In [1]:
from train_data_processor import TrainDataProcessor
from test_data_processor import TestDataProcessor
import pandas as pd

In [2]:
# Train Datasının Çağırılması
train_file_path = "data/train.csv"
train_data_processor = TrainDataProcessor(train_file_path)
train_df,columns_to_dropped = train_data_processor.run_process()

In [3]:
# Test Datasının Çağırılması
test_file_path = "data/test_x.csv"
test_data_processor = TestDataProcessor(test_file_path)
test_df = test_data_processor.run_process()

before 11049
after 11049


In [4]:
train_df.drop(columns=columns_to_dropped, inplace=True)
test_df.drop(columns=columns_to_dropped, inplace=True)
columns_to_drop = ["Universite Adi","Universite Kategori",'Lise Adi','Lise Sehir']
train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

In [5]:
# Test STK Proje sütunun tahmini.
train_df_training = train_df[['Aktif olarak bir STK üyesi misiniz?', 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?','Girisimcilikle Ilgili Deneyiminiz Var Mi?','Age',"Stk Projesine Katildiniz Mi?"]]
X = train_df_training.drop('Stk Projesine Katildiniz Mi?', axis=1)
y = train_df_training['Stk Projesine Katildiniz Mi?']
test_df_training = test_df[['Aktif olarak bir STK üyesi misiniz?', 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?','Girisimcilikle Ilgili Deneyiminiz Var Mi?','Age',"Stk Projesine Katildiniz Mi?"]]
test_df_training = test_df_training.drop(["Stk Projesine Katildiniz Mi?"],axis=1)
## Doldurulmuş Test Verisini kaydetme.
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6,verbose=100)

# Modeli eğit
model.fit(X, y)
# Test verisinde tahmin yapma
y_pred = model.predict(test_df_training)
test_df["Stk Projesine Katildiniz Mi?"] = y_pred

0:	learn: 0.6754364	total: 163ms	remaining: 1m 21s
100:	learn: 0.6047150	total: 1.05s	remaining: 4.17s
200:	learn: 0.6043330	total: 2.14s	remaining: 3.19s
300:	learn: 0.6042405	total: 3.08s	remaining: 2.04s
400:	learn: 0.6042061	total: 3.92s	remaining: 967ms
499:	learn: 0.6041931	total: 4.58s	remaining: 0us


In [6]:
# NaN'ların doldurulması.
from sklearn.impute import KNNImputer

# KNN Imputer kullanarak NaN değerlerini doldurma
train_imputer = KNNImputer(n_neighbors=5)
train_df = pd.DataFrame(train_imputer.fit_transform(train_df), columns=train_df.columns)

# KNN Imputer kullanarak NaN değerlerini doldurma
test_imputer = KNNImputer(n_neighbors=5)
test_df = pd.DataFrame(test_imputer.fit_transform(test_df), columns=test_df.columns)

In [7]:
train_df.loc[train_df['Basvuru Yili'] == 2016, 'Degerlendirme Puani'] = train_df.loc[train_df['Basvuru Yili'] == 2016, 'Degerlendirme Puani'] / 2
train_df['Basvuru Yili'] = train_df['Basvuru Yili'] - 2000

# test_df.loc[test_df['Basvuru Yili'] == 2016, 'Degerlendirme Puani'] = test_df.loc[test_df['Basvuru Yili'] == 2016, 'Degerlendirme Puani'] / 2
test_df['Basvuru Yili'] = test_df['Basvuru Yili'] - 2000

In [8]:
from catboost import CatBoostRegressor
from catboost import Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Veri setini özellikler (X) ve hedef değişken (y) olarak ayırma
X = train_df.drop('Degerlendirme Puani', axis=1)
y = train_df['Degerlendirme Puani']
# CatBoostRegressor modeli oluşturma
model = CatBoostRegressor(
    iterations=1000,              # Ağaç sayısı
    learning_rate=0.05,           # Öğrenme oranı
    depth=8,                     # Ağaç derinliği
    l2_leaf_reg = 5,
    # cat_features=categorical_features, # Kategorik özellikler
    random_seed=42,
    eval_metric='RMSE'           # Performans metriği
)
# Modeli eğitme
model.fit(X, y,verbose=100) #  verbose=100
# Model ile tahmin yapma
y_pred = model.predict(test_df)

0:	learn: 14.1506061	total: 24ms	remaining: 24s
100:	learn: 4.8279301	total: 1.11s	remaining: 9.87s
200:	learn: 4.4867361	total: 2.15s	remaining: 8.54s
300:	learn: 4.3101374	total: 3.14s	remaining: 7.29s
400:	learn: 4.1850620	total: 4.15s	remaining: 6.2s
500:	learn: 4.0878219	total: 5.3s	remaining: 5.28s
600:	learn: 4.0052432	total: 6.32s	remaining: 4.19s
700:	learn: 3.9251382	total: 7.39s	remaining: 3.15s
800:	learn: 3.8539828	total: 8.47s	remaining: 2.1s
900:	learn: 3.7874549	total: 9.57s	remaining: 1.05s
999:	learn: 3.7253969	total: 10.6s	remaining: 0us


In [9]:
prediction_df = pd.DataFrame()
prediction_df["id"] = range(0,len(y_pred))
prediction_df["Degerlendirme Puani"] = y_pred

In [10]:
prediction_df

,id,Degerlendirme Puani
0,0,29.327792
1,1,22.126579
2,2,5.142465
3,3,20.368860
4,4,40.927279
...,...,...
11044,11044,31.516964
11045,11045,30.233800
11046,11046,43.984083
11047,11047,45.264299


In [19]:
prediction_df.to_csv("first_step_newestt3.csv",index=False)